# Подготовка файла с данными для создания дашборда

`Задача`: получить и подготовить данные для создания дашборда взаимодействия пользователей с карточками статей Яндекс.Дзен.  

Состав данных для дашборда:  
- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);  
- Разбивка событий по темам источников;  
- Таблица соответствия тем источников темам карточек;

## Подключение к базе данных, запрос данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'xxx', # имя пользователя
            'pwd': 'xxx', # пароль
            'host': 'xxx.yandexcloud.net',
            'port': 6666, # порт подключения
            'db': 'xxx'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

query = ''' SELECT *
            FROM dash_visits
        '''


dash_visits = pd.io.sql.read_sql(query, con = engine) 

dash_visits

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
...,...,...,...,...,...,...
30740,1071337,Юмор,Финансы,36-40,2019-09-24 18:57:00,2
30741,1071338,Юмор,Финансы,36-40,2019-09-24 19:00:00,1
30742,1071339,Юмор,Финансы,41-45,2019-09-24 18:54:00,1
30743,1071340,Юмор,Финансы,41-45,2019-09-24 18:56:00,1


## Обработка полученых данных для последующего создания дашборда на их основе

In [2]:
dash_visits['item_topic'].unique()

array(['Деньги', 'Дети', 'Женская психология', 'Женщины', 'Здоровье',
       'Знаменитости', 'Интересные факты', 'Искусство', 'История',
       'Красота', 'Культура', 'Наука', 'Общество', 'Отношения',
       'Подборки', 'Полезные советы', 'Психология', 'Путешествия',
       'Рассказы', 'Россия', 'Семья', 'Скандалы', 'Туризм', 'Шоу', 'Юмор'],
      dtype=object)

In [3]:
dash_visits['source_topic'].unique()

array(['Авто', 'Деньги', 'Дети', 'Еда', 'Здоровье', 'Знаменитости',
       'Интерьеры', 'Искусство', 'История', 'Кино', 'Музыка', 'Одежда',
       'Полезные советы', 'Политика', 'Психология', 'Путешествия',
       'Ремонт', 'Россия', 'Сад и дача', 'Сделай сам',
       'Семейные отношения', 'Семья', 'Спорт', 'Строительство',
       'Технологии', 'Финансы'], dtype=object)

In [4]:
dash_visits.to_csv('dash_visits.csv')

In [5]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [11]:
dash_visits['dt'].describe()

/tmp/ipykernel_714/2972088052.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  dash_visits['dt'].describe()


count                   30745
unique                     17
top       2019-09-24 18:58:00
freq                     3383
first     2019-09-24 18:28:00
last      2019-09-24 19:00:00
Name: dt, dtype: object

In [7]:
dash_visits['item_topic'].value_counts().reset_index()

,index,item_topic
0,Отношения,1536
1,Интересные факты,1535
2,Наука,1505
3,Подборки,1456
4,Полезные советы,1424
5,Общество,1422
6,Россия,1385
7,История,1363
8,Семья,1287
9,Путешествия,1247


In [8]:
dash_visits_grouped = dash_visits.groupby(['dt', 'item_topic', 'source_topic', 'age_segment']).agg(
    {'record_id':'count', 'visits':'sum'}
).reset_index()

In [9]:
dash_visits_grouped.sample(20)

,dt,item_topic,source_topic,age_segment,record_id,visits
24186,2019-09-24 18:58:00,Семья,Сделай сам,18-25,1,39
11353,2019-09-24 18:54:00,Туризм,Психология,45+,1,2
16128,2019-09-24 18:56:00,Наука,Искусство,26-30,1,28
30145,2019-09-24 19:00:00,Рассказы,Строительство,41-45,1,1
18903,2019-09-24 18:57:00,Искусство,Деньги,18-25,1,12
29197,2019-09-24 19:00:00,Наука,Здоровье,26-30,1,6
2890,2019-09-24 18:30:00,Юмор,Путешествия,45+,1,1
9663,2019-09-24 18:54:00,Интересные факты,Россия,26-30,1,28
5038,2019-09-24 18:32:00,Психология,Знаменитости,18-25,1,3
19134,2019-09-24 18:57:00,История,Сделай сам,36-40,1,5


Выгрузка данных в файл для работы в Tableau

In [10]:
dash_visits_grouped.to_csv('dash_visits_grouped.csv')

Данные готовы к созданию дашборда